In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_4420/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.9)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_12_9_14,0.998924,0.995051,0.997577,0.996319,0.129802,0.468657,0.259764,0.370354,"Hidden Size=[35], regularizer=0.1, learning_ra..."
1,model_12_9_15,0.998922,0.994892,0.997504,0.996204,0.130027,0.483668,0.267571,0.381975,"Hidden Size=[35], regularizer=0.1, learning_ra..."
2,model_12_9_13,0.998921,0.995221,0.997654,0.996442,0.130117,0.452569,0.251528,0.357961,"Hidden Size=[35], regularizer=0.1, learning_ra..."
3,model_12_9_16,0.998917,0.994745,0.997436,0.996096,0.130642,0.497599,0.274919,0.392809,"Hidden Size=[35], regularizer=0.1, learning_ra..."
4,model_12_9_12,0.998912,0.995402,0.997734,0.996573,0.131162,0.435413,0.242906,0.344822,"Hidden Size=[35], regularizer=0.1, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...
1057,model_12_6_6,0.964628,0.945849,0.979859,0.956897,4.265928,8.999774,1.727642,5.577594,"Hidden Size=[35], regularizer=0.1, learning_ra..."
1158,model_12_6_5,0.959768,0.946499,0.980162,0.957434,4.851956,8.891724,1.701614,5.508142,"Hidden Size=[35], regularizer=0.1, learning_ra..."
1166,model_13_7_5,0.959038,0.967541,0.934622,0.948472,4.939978,3.431686,14.369580,8.578931,"Hidden Size=[10, 5], regularizer=0.01, learnin..."
1169,model_13_7_4,0.958853,0.972821,0.948558,0.958905,4.962344,2.873499,11.306581,6.842008,"Hidden Size=[10, 5], regularizer=0.01, learnin..."
